In [1]:
import numpy as np
from numpy import pi
# importing Qiskit
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy import linalg as la
import tensorflow as tf

In [4]:
def real(qc, phi, theta, omega):
    # Adds the real data to the quantum circuit
    qc.h(0)
    qc.rx(phi,0)
    qc.ry(theta,0)
    qc.rz(omega,0)

def generator(qc, w):
    qc.rx(w[0],0)
    qc.rx(w[1],1)
    qc.ry(w[2],0)
    qc.ry(w[3],1)
    qc.rz(w[4],0)
    qc.rz(w[5],1)
    qc.cnot(0,1)
    qc.rx(w[6],0)
    qc.ry(w[7],0)
    qc.rz(w[8],0)
    
def discriminator(qc, w):
    qc.h(0)
    qc.rx(w[0],0)
    qc.rx(w[1],2)
    qc.ry(w[2],0)
    qc.ry(w[3],2)
    qc.rz(w[4],0)
    qc.rz(w[5],2)
    qc.cnot(0,2)
    qc.rx(w[6],2)
    qc.ry(w[7],2)
    qc.rz(w[8],2)
    qc.measure(2,0)

In [9]:
def real_disc_circuit(phi, theta, omega, disc_weights):
    qc = QuantumCircuit(3, 1)
    real(qc, phi, theta, omega)
    discriminator(qc, disc_weights)
    return qc

def gen_disc_circuit(gen_weights, disc_weights):
    qc = QuantumCircuit(3, 1)
    generator(qc, gen_weights)
    discriminator(qc, disc_weights)
    return qc

In [10]:
shots = 2**14
sim = Aer.get_backend('aer_simulator')

phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

In [11]:
def disc_cost(disc_weights):
    qc = real_disc_circuit(phi,theta,omega,disc_weights)
    circ_trans = transpile(qc.compose(qc), sim)
    counts = sim.run(qc.compose(circ_trans), shots=shots).result().get_counts()
    prob_disc = counts["1"]/(counts['0']+counts["1"])
    
    qc = gen_disc_circuit(gen_weights,disc_weights)
    circ_trans = transpile(qc.compose(qc), sim)
    counts = sim.run(qc.compose(circ_trans), shots=shots).result().get_counts()
    prob_gen = counts["1"]/(counts['0']+counts["1"])
    return prob_gen - prob_disc

def gen_cost(gen_weights):
    qc = gen_disc_circuit(gen_weights,disc_weights)
    circ_trans = transpile(qc.compose(qc), sim)
    counts = sim.run(qc.compose(circ_trans), shots=shots).result().get_counts()
    prob_gen = counts["1"]/(counts['0']+counts["1"])
    return -prob_gen

In [13]:
opt = tf.keras.optimizers.SGD(0.4)

In [15]:
print(gen_weights[0])

tf.Tensor(3.1592331770494697, shape=(), dtype=float64)


In [14]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

CircuitError: "Invalid param type <class 'tensorflow.python.framework.ops.EagerTensor'> for gate rx."